In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('YOLO').getOrCreate()
print("Spark object is created")
print("Number of partitions for shuffle changed to : " + str(spark.conf.get('spark.sql.shuffle.partitions')))

Spark object is created
Number of partitions for shuffle changed to : 200


In [2]:
ord = spark.sparkContext.textFile("F:\digi-world\pyspark\data-all\PracticeFiles\Orders\part-00000")
ord_item = spark.sparkContext.textFile("F:\digi-world\pyspark\data-all\PracticeFiles\Order_items\part-00000")

In [23]:
for i in ord.take(5): print(i)
print()
for j in ord_item.take(5): print(j)

1,2013-07-25 00:00:00.0,11599,CLOSED
2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
3,2013-07-25 00:00:00.0,12111,COMPLETE
4,2013-07-25 00:00:00.0,8827,CLOSED
5,2013-07-25 00:00:00.0,11318,COMPLETE

1,1,957,1,299.98,299.98
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99


***Joins***

In [37]:
print("Orders:")
ordMap = ord.map(lambda x : (x.split(',')[0], x.split(',')[2]))
for i in ordMap.take(5):    print(i)
print()
print("Order Items:")
ordItemsMap=ord_item.map(lambda x : (x.split(',')[1],x.split(',')[4]))
for i in ordItemsMap.take(5):   print(i)

Orders:
('1', '11599')
('2', '256')
('3', '12111')
('4', '8827')
('5', '11318')

Order Items:
('1', '299.98')
('2', '199.99')
('2', '250.0')
('2', '129.99')
('4', '49.98')


In [46]:
findSubtotalForCust = ordMap.join(ordItemsMap)
for i in findSubtotalForCust.take(10): print(i)

('1', ('11599', '299.98'))
('4', ('8827', '49.98'))
('4', ('8827', '299.95'))
('4', ('8827', '150.0'))
('4', ('8827', '199.92'))
('8', ('2911', '179.97'))
('8', ('2911', '299.95'))
('8', ('2911', '199.92'))
('8', ('2911', '50.0'))
('9', ('5657', '199.98'))


In [44]:
findSubtotalForCust.map(lambda x : str(x[1][0])+','+str(x[1][1])).first()

'11599,299.98'

***CoGroup***

In [60]:
x = spark.sparkContext.parallelize([("a", 1), ("b", 4)])
y = spark.sparkContext.parallelize([("a", 2)])
xy = x.cogroup(y)
xy.take(3)

[('b',
  (<pyspark.resultiterable.ResultIterable at 0x284eb7a0b08>,
   <pyspark.resultiterable.ResultIterable at 0x284eb7a0148>)),
 ('a',
  (<pyspark.resultiterable.ResultIterable at 0x284eb7a4e08>,
   <pyspark.resultiterable.ResultIterable at 0x284eb7a4888>))]

In [67]:
for i,j in xy.take(2) : print(str(i)+' > '+ str(list(map(list, j))))

b > [[4], []]
a > [[1], [2]]


***Cartesian***

In [70]:
rdd = spark.sparkContext.parallelize((1,3,2))
sorted(rdd.cartesian(rdd).collect(), reverse=False)

[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]